In [ ]:
# == 모델 실행 == 
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch
from transformers import PreTrainedTokenizerFast
from torch.utils.data import TensorDataset, DataLoader
from transformers import AdamW
import pandas as pd

# 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("./food_chatbot_model_최종")
model = GPT2LMHeadModel.from_pretrained("./food_chatbot_model_최종")
model.to(device)
model.eval()

def generate_response(input_text):
    # 입력 길이 제한 (512자까지 허용)
    input_text = input_text[:512]
    
    # 입력 텍스트 인코딩
    input_ids = tokenizer.encode(f"질문: {input_text}\n답변: ", return_tensors='pt').to(device)

    # 답변 생성
    with torch.no_grad():
        output_ids = model.generate(
            input_ids,
            max_length=200,  # 질문 길이 + 50자
            min_length=20,  # 최소 답변 길이
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.92,
            temperature=0.7,
            repetition_penalty=1.2,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=True
        )

    # 출력 디코딩
    output = tokenizer.decode(
        output_ids[0],
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )

    # '답변:' 이후의 텍스트 추출
    try:
        response = output.split("답변:")[1].strip()
    except IndexError:
        response = "죄송해요, 답변을 생성하는 데 문제가 발생했어요."

    return response

In [ ]:
model.to(device)
model.eval()

print("음식 지식 챗봇입니다. 종료하려면 '종료'를 입력하세요.")

while True:
    user_input = input("사용자: ")
    if user_input.strip().lower() == "종료":
        print("챗봇: 대화를 종료합니다. 좋은 하루 되세요!")
        break
    response = generate_response(user_input)
    print(f"챗봇: {response}")

In [1]:
import torch

print(torch.cuda.is_available())


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\ProgramData\anaconda3\envs\ml-dev\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\ProgramData\anaconda3\envs\ml-dev\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\ProgramData\anaconda3\envs\ml-dev\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\ProgramData\anaconda3\envs\ml-dev\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start

False


In [2]:
import torch

# GPU 사용 가능 여부 확인
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU 사용 중:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU를 사용하지 않고 있습니다. CPU 사용 중")

GPU 사용 중: NVIDIA GeForce GTX 1660 SUPER
